### Generate Candlestick chart using historical OHLC data

In [18]:
import plotly.graph_objects as go
import plotly.io as pio
import argparse
import pandas as pd

pio.renderers.default = 'browser'
display_data = 2000

ohlc_df = pd.read_csv(f'../data/btc.csv', nrows=display_data)
fig = go.Figure(
    data=[go.Candlestick(
        x=ohlc_df['Timestamp'], 
        open=ohlc_df['Open'], 
        high=ohlc_df['High'], 
        low=ohlc_df['Low'], 
        close=ohlc_df['Close'],
        increasing_line_color= '#29d691',
        decreasing_line_color= '#ee113e'
    )], 
)

last_ohlc_timestamp = ohlc_df['Timestamp'].iloc[-1]

### Plot all closed positions using gray ⚪ for entry time and either 🟢 if PnL > 0 or 🔴 otherwise

In [19]:
positions_df = pd.read_csv('../results/closed_positions.csv', nrows=display_data // 2)
position_id = 1

for index, row in positions_df.iterrows():
    if row['Exit'] > last_ohlc_timestamp:
        break

    color = 'green' if int(row['PnL']) > 0 else 'red'
    entry_low = ohlc_df.loc[ohlc_df['Timestamp'] == row['Entry'], 'Low'].values[0]
    exit_low = ohlc_df.loc[ohlc_df['Timestamp'] == row['Exit'], 'Low'].values[0]

    fig.add_trace(go.Scatter(
        x=[row['Entry']],
        y=[entry_low],
        mode='markers',
        marker=dict(
            size=22, 
            color='rgba(131, 131, 131, 0.7)', 
            symbol='circle'
        ),
        name='Entry',
        text=[f"Entry: {row['Entry']}<br>Position ID: {position_id}<br>Type: {row['Type'].capitalize()}"],
        hoverinfo='text'
    ))

    fig.add_trace(go.Scatter(
        x=[row['Exit']],
        y=[exit_low],
        mode='markers',
        marker=dict(
            size=22,
            symbol='circle',
            color=['rgba(0, 200, 0, 0.4)' if row['PnL'] > 0 else 'rgba(255, 0, 0, 0.4)']
        ),
        name='Exit',
        text=[f"Exit: {row['Exit']}<br>Position ID: {position_id}<br>Type: {row['Type'].capitalize()}<br>PnL: {row['PnL']}"],
        hoverinfo='text'
    ))

    position_id += 1

### Plot all trades using 🟢▲ for Sell trades and 🔴▼ for Buy trades (optional)

In [20]:
trades_df = pd.read_csv('../results/trades.csv', nrows=display_data)

for index, row, in trades_df.iterrows():
    if row['Timestamp'] > last_ohlc_timestamp:
        break

    color = 'green' if row['Side'] == 'Sell' else 'red'
    fig.add_trace(go.Scatter(
        x=[row['Timestamp']],
        y=[row['Price']],
        mode='markers',
        name=row['Side'],
        marker=dict(
            symbol=('triangle-down' if row['Side'] == 'Buy' else 'triangle-up'), 
            size=10, 
            color=color
        ),
        hovertemplate=f"{row['Side']}<br>Price: ${row['Price']}<br>Quantity: {row['Quantity']}<br>Time: {row['Timestamp']}",
    ))

### Plot Bollinger Bands indicator on visualisation (optional)

In [21]:
bollinger_bands_df = pd.read_csv('../results/bollinger_bands.csv', nrows=display_data)
filtered_bollinger = bollinger_bands_df[bollinger_bands_df['Timestamp'] <= last_ohlc_timestamp]

fig.add_trace(go.Scatter(
    x=filtered_bollinger['Timestamp'], 
    y=filtered_bollinger['Lower'],
    mode='lines',
    name='Bollinger Lower Band',
    line=dict(color='rgba(54, 162, 235, 0.8)', width=1.5, dash='dot')
))

fig.add_trace(go.Scatter(
    x=filtered_bollinger['Timestamp'], 
    y=filtered_bollinger['Middle'],
    mode='lines',
    name='Moving Average (SMA)',
    line=dict(color='rgba(0, 0, 0, 0.6)', width=1.5)
))

fig.add_trace(go.Scatter(
    x=filtered_bollinger['Timestamp'], 
    y=filtered_bollinger['Upper'],
    mode='lines',
    name='Bollinger Upper Band',
    line=dict(color='rgba(54, 162, 235, 0.8)', width=1.5, dash='dot')
))

### Render the visualisation

In [22]:
fig.update_layout(template="ggplot2", title=dict(text="Trading Strategy Performance Visualisation"))
fig.show()